In [1]:
import pretty_midi
from moviepy.editor import *
import glob

In [2]:
file_name = '../songs/guren.mid'
SRC_PATH = "../new_src"

In [3]:
midi_data = pretty_midi.PrettyMIDI(file_name)

In [4]:
print("duration:",midi_data.get_end_time())

duration: 100.31182558333343


In [5]:
def find_min_start(notes):
    min_notes = []
    pop_index = []
    if len(notes) > 0:
        min_note = notes[0]
        min_start = min_note.start
        for i in range(len(notes)):
            note = notes[i]
            if note.start == min_start:
                min_notes.append(note)
                pop_index.append(i)
            else:
                break
        for i in range(len(pop_index)):
            index = pop_index[len(pop_index) - i - 1]
            notes.pop(index)
    return min_notes

In [6]:
def find_first_end(notes):
    if len(notes) > 0:
        min_end = notes[0].end
        for note in notes:
            if note.end < min_end:
                min_end = note.end
        return min_end
    return -1

In [7]:
def find_before(end, notes):
    if len(notes) > 0:
        min_note = notes[0]
        min_start = min_note.start
        if min_start > end:
            return []
        else:
            return find_min_start(notes)
    return []

In [8]:
def check_next_playing_note(playing_notes, new_start_note, current_time):
    note_still_alive = []
    for note in playing_notes[-1]:
        if note.end > current_time:
            note_still_alive.append(note)
    for note in new_start_note:
        note_still_alive.append(note)
    return note_still_alive

In [9]:
def split_note_frame(notes):
    playing_notes = []
    start_time = []
    end_time = []
    notes_start_before_end = []
    current_notes = []
    current_time = 0
    
    while len(notes) > 0 or len(current_notes) > 0:
        
        if len(current_notes) < 1:
            # get current notes
            current_notes = find_min_start(notes)
            
        # add current notes to playing notes list
        playing_notes.append(current_notes)

        # set start time
        start_time.append(current_notes[0].start)

        # get first end in current playing
        end = find_first_end(current_notes)

        # find notes that start before playing notes has end
        notes_start_before_end = find_before(end, notes)


        # if have note start before end then cut as end_time
        if len(notes_start_before_end) > 0:
            end_time.append(notes_start_before_end[0].start)
            
        # if not have note start then it's end by time out
        else:
            end_time.append(end)

        # get current time to check time out
        current_time = end_time[-1]

        # get playing note, remove time out note and merge new note
        current_notes = check_next_playing_note(playing_notes, notes_start_before_end, current_time)

    return playing_notes, start_time, end_time

In [10]:
len(midi_data.instruments)

2

In [11]:
notes = []

In [12]:
for ins in midi_data.instruments:
    notes += ins.notes

In [13]:
notes.sort(key=lambda x: x.start)

In [14]:
notes

[Note(start=2.655389, end=2.828041, pitch=61, velocity=93),
 Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
 Note(start=2.655389, end=3.321135, pitch=49, velocity=99),
 Note(start=2.828041, end=2.983428, pitch=61, velocity=88),
 Note(start=2.993097, end=3.146412, pitch=64, velocity=89),
 Note(start=3.156771, end=3.486191, pitch=63, velocity=88),
 Note(start=3.321135, end=3.813539, pitch=56, velocity=91),
 Note(start=3.486191, end=3.649865, pitch=59, velocity=88),
 Note(start=3.649865, end=3.803871, pitch=59, velocity=89),
 Note(start=3.813539, end=4.144341, pitch=57, velocity=80),
 Note(start=3.813539, end=4.154009, pitch=61, velocity=90),
 Note(start=3.813539, end=3.967545, pitch=45, velocity=88),
 Note(start=3.979285, end=4.975833, pitch=52, velocity=88),
 Note(start=4.154009, end=4.309396, pitch=61, velocity=88),
 Note(start=4.319065, end=4.472380, pitch=64, velocity=89),
 Note(start=4.482739, end=4.812159, pitch=63, velocity=88),
 Note(start=4.812159, end=5.303872, pitc

In [15]:
len(notes)

1106

In [16]:
results, start , end = split_note_frame(notes)

In [17]:
results

[[Note(start=2.655389, end=2.828041, pitch=61, velocity=93),
  Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
  Note(start=2.655389, end=3.321135, pitch=49, velocity=99)],
 [Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
  Note(start=2.655389, end=3.321135, pitch=49, velocity=99),
  Note(start=2.828041, end=2.983428, pitch=61, velocity=88)],
 [Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
  Note(start=2.655389, end=3.321135, pitch=49, velocity=99)],
 [Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
  Note(start=2.655389, end=3.321135, pitch=49, velocity=99),
  Note(start=2.993097, end=3.146412, pitch=64, velocity=89)],
 [Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
  Note(start=2.655389, end=3.321135, pitch=49, velocity=99)],
 [Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
  Note(start=2.655389, end=3.321135, pitch=49, velocity=99),
  Note(start=3.156771, end=3.486191, pitch=63, velocity=88)],
 [Note(start=3.156

In [18]:
def get_clip(note_number):
    path_clip = os.path.join(SRC_PATH, str(note_number) + ".mp4")
    clip = VideoFileClip(path_clip)
    return clip

In [23]:
def path_to_number(path, only_key = False):
    files = glob.glob(os.path.join(path, "*"))
    clips = {}
    for f in files:
        number = os.path.basename(f).split(".")[0]
        if only_key:
            clips[number] = f
        else:
            clips[number] = get_clip(number)
    return clips

# Test Load Clip

In [20]:
c = get_clip(results[0][0].pitch)

In [21]:
c.duration

3.34

In [22]:
c.ipython_display()

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [22]:
sound_clips = []

### Warning Next process will take some times and memory !!!

In [24]:
clips = path_to_number(SRC_PATH)

In [25]:
clips

{'25': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eaefe1748>,
 '27': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f4da90>,
 '28': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5a630>,
 '29': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5ac88>,
 '30': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5a748>,
 '31': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5a860>,
 '32': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5a400>,
 '33': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5a588>,
 '34': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5ab70>,
 '35': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f5af28>,
 '36': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f690b8>,
 '37': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f69208>,
 '38': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f69400>,
 '39': <moviepy.video.io.VideoFileClip.VideoFileClip at 0x24eb2f

In [26]:
video_clips = []

In [27]:
for i in range(len(results)):
    notes = results[i]
    start_time = start[i]
    end_time = end[i]
    for note in notes:
        clip = clips[str(note.pitch)].subclip(0.5)
        diff_time= end_time - start_time
        clip = clip.set_start(start_time)

        if diff_time <= clip.duration:
            clip = clip.set_end(end_time)
        
        video_clips.append(clip)
        
        # uncomment this if you want to process with all clips
        break

In [28]:
len(video_clips)

946

In [29]:
len(results)

946

In [30]:
video = CompositeVideoClip(video_clips)

In [31]:
video.write_videofile("../results/guren_my_voice_video_only.mp4", threads=4, audio = False)

t:   0%|                                                                    | 4/5988 [00:00<02:37, 38.10it/s, now=None]

Moviepy - Building video ../results/guren_my_voice_video_only.mp4.
Moviepy - Writing video ../results/guren_my_voice_video_only.mp4



Moviepy - Done !
Moviepy - video ready ../results/guren_my_voice_video_only.mp4


In [32]:
len(notes)

5

In [33]:
notes = []

In [34]:
for ins in midi_data.instruments:
    notes += ins.notes

In [35]:
notes.sort(key=lambda x: x.start)

In [36]:
len(notes)

1106

In [38]:
sound_clips = []

In [39]:
for i in range(len(notes)):
    note = notes[i]
    start_time = note.start
    end_time = note.end
    
    clip = clips[str(note.pitch)].subclip(0.5)
    diff_time= end_time - start_time
    clip = clip.set_start(start_time)
    
#     if diff_time <= clip.duration:
#         clip = clip.set_end(end_time)
    
    sound_clips.append(clip.audio)

In [40]:
audio = CompositeAudioClip(sound_clips)

In [41]:
sound_clips[0].fps

44100

In [42]:
audio

In [43]:
audio.write_audiofile("../results/only_sound.mp3", fps=44100)

chunk:   0%|▎                                                               | 9/2203 [00:00<00:25, 84.92it/s, now=None]

MoviePy - Writing audio in ../results/only_sound.mp3


MoviePy - Done.


# merge

In [44]:
video_only = VideoFileClip("../results/guren_my_voice_video_only.mp4")
audio_only = AudioFileClip("../results/only_sound.mp3")

In [45]:
video_only.duration

99.9

In [46]:
audio_only.duration

99.94

In [47]:
video_only = video_only.set_audio(audio_only)

In [48]:
video_only.write_videofile("../results/merged_my_voice.mp4")

chunk:   3%|█▋                                                            | 60/2204 [00:00<00:03, 594.15it/s, now=None]

Moviepy - Building video ../results/merged_my_voice.mp4.
MoviePy - Writing audio in merged_my_voiceTEMP_MPY_wvf_snd.mp3


t:   0%|                                                                    | 5/5989 [00:00<02:03, 48.56it/s, now=None]

MoviePy - Done.
Moviepy - Writing video ../results/merged_my_voice.mp4



Moviepy - Done !
Moviepy - video ready ../results/merged_my_voice.mp4
